In [1]:
from selenium import webdriver
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
from selenium.common.exceptions import InvalidArgumentException
import requests
from bs4 import BeautifulSoup
import urllib.robotparser
import numpy as np
import pandas as pd

In [2]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.chotot.com/mua-ban-cho-tp-ho-chi-minh/robots.txt')
rp.read()

url = 'https://www.chotot.com/mua-ban-cho-tp-ho-chi-minh'
rp.can_fetch('*', url)

False

In [3]:
def crawl_one_dog(link):
    print('https://www.chotot.com' + link)
    driver = webdriver.Firefox()
    try:
        driver.get('https://www.chotot.com' + link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        price = np.nan
        description = np.nan
        breed = np.nan        
        age = np.nan        
        size = np.nan
        address = np.nan
        post_time = np.nan
        # phone_num = np.nan

        post = soup.find('span', attrs = {'class': 'AdImage_imageCaptionText__ScM56'})
        post_time = post.text if post else np.nan 

        price_tag = soup.find('span', attrs = {'itemprop': 'price'})
        price = price_tag.text if price_tag else np.nan

        description_tag = soup.find('p', attrs = {'class': 'AdDecription_adBody__qp2KG'})
        description = description_tag.text if description_tag else np.nan

        breed_tag = soup.find('span', attrs = {'itemprop': 'pet_breed'})
        breed = breed_tag.text if breed_tag else np.nan

        age_tag = soup.find('span', attrs = {'itemprop': 'pet_age'})
        age = age_tag.text if age_tag else np.nan

        size_tag = soup.find('span', attrs = {'itemprop': 'pet_size'})
        size = size_tag.text if size_tag else np.nan

        address_tag = soup.find('div', attrs={'class':'media-body media-middle AdParam_address__5wp1F'})
        address = address_tag.text if address_tag else np.nan

        # # Click the button to show the phone number
        # driver.execute_script("var element = document.querySelector('.aw__s1cdo2zu'); if (element) element.parentNode.removeChild(element);")
        # button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CLASS_NAME, 'ShowPhoneButton_icon__wsnZ5')))
        # driver.execute_script("arguments[0].scrollIntoView();", button)
        # button.click()
        # soup = BeautifulSoup(driver.page_source, 'html.parser')
        # # Wait for the phone number to load (adjust the timeout as needed)
        # phone_num_tag = soup.find('div', attrs = {'class': 'ShowPhoneButton_phoneButton__p5Cvt ShowPhoneButton_phoneClicked__IxuR6'})
        # phone_num = phone_num_tag.text if phone_num_tag else np.nan

        driver.quit()
        return post_time, price, description, breed, age, size, address
    except InvalidArgumentException:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    finally:
        driver.quit()

In [4]:
df = pd.DataFrame()
param = '?page='
# index_page = 123
index_page = 1
while(True):
    driver = webdriver.Firefox()

    driver.get(url + param + str(index_page))
    print(url + param + str(index_page))
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    not_found = soup.find('div', attrs={'class': 'NotFound_content__KtIbC'})
    if not_found is not None:
        print('No more dogs to crawl')
        break
    
    dogs_links = soup.find_all('a', attrs={'class': 'AdItem_adItem__gDDQT'})
    # item_footers = soup.find_all('span', attrs={'class': 'AdItemFooter_item__v9Cg0'})
    # post = [item.text if item else np.nan for item in item_footers]

    driver.quit()

    for i in dogs_links:
        post_time, price, description, breed, age, size, address = crawl_one_dog(i['href'])
            
        data = pd.DataFrame({
                'post_time' : [post_time],
                'price': [price],
                'description': [description],
                'breed': [breed],
                'age': [age],
                'size': [size],
                'address': [address],
                # 'phone_num': [phone_num]
            })
        df = pd.concat([df, data], axis = 0)    

    index_page += 1

https://www.chotot.com/mua-ban-cho-tp-ho-chi-minh?page=1
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/111629482.htm#px=SR-stickyad-[PO-1][PL-top]
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/117146149.htm#px=SR-stickyad-[PO-2][PL-top]
https://www.chotot.com/mua-ban-cho-quan-8-tp-ho-chi-minh/115905112.htm#px=SR-stickyad-[PO-3][PL-top]
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/88243216.htm#px=SR-stickyad-[PO-4][PL-top]
https://www.chotot.com/mua-ban-cho-quan-8-tp-ho-chi-minh/115905075.htm#px=SR-stickyad-[PO-5][PL-top]
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/103804726.htm
https://www.chotot.com/mua-ban-cho-quan-8-tp-ho-chi-minh/115905187.htm
https://www.chotot.com/mua-ban-cho-huyen-cu-chi-tp-ho-chi-minh/117556312.htm
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/115158520.htm
https://www.chotot.com/mua-ban-cho-quan-go-vap-tp-ho-chi-minh/117507939.htm
https://www.chotot.com/mua-ban-cho-quan-

https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/111629504.htm
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/111143924.htm
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/91764649.htm
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/117470154.htm
https://www.chotot.com/mua-ban-cho-quan-6-tp-ho-chi-minh/117548795.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/117503731.htm
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/102311728.htm
https://www.chotot.com/mua-ban-cho-quan-3-tp-ho-chi-minh/111012997.htm#px=SR-special_display_ad-[PO-17][PL-default]
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/91185632.htm
https://www.chotot.com/mua-ban-cho-huyen-cu-chi-tp-ho-chi-minh/117548515.htm
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/117316894.htm
https://www.chotot.com/mua-ban-cho-tp-ho-chi-minh?page=6
https://www.chotot.com/mua-ban-cho-quan-go-vap-tp-ho-chi-minh

https://www.chotot.com/mua-ban-cho-quan-go-vap-tp-ho-chi-minh/117539178.htm
https://www.chotot.com/mua-ban-cho-quan-8-tp-ho-chi-minh/117539174.htm
https://www.chotot.com/mua-ban-cho-huyen-binh-chanh-tp-ho-chi-minh/115973441.htm
https://www.chotot.com/mua-ban-cho-quan-7-tp-ho-chi-minh/117539104.htm
https://www.chotot.com/mua-ban-cho-huyen-binh-chanh-tp-ho-chi-minh/117161800.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/117538460.htm
https://www.chotot.com/mua-ban-cho-huyen-binh-chanh-tp-ho-chi-minh/116232165.htm
https://www.chotot.com/mua-ban-cho-huyen-hoc-mon-tp-ho-chi-minh/117538323.htm
https://www.chotot.com/mua-ban-cho-quan-10-tp-ho-chi-minh/109919321.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/117462027.htm#px=SR-special_display_ad-[PO-18][PL-default]
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/116714762.htm#px=SR-special_display_ad-[PO-19][PL-default]
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-

https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/102427012.htm
https://www.chotot.com/mua-ban-cho-quan-11-tp-ho-chi-minh/116464686.htm
https://www.chotot.com/mua-ban-cho-huyen-cu-chi-tp-ho-chi-minh/117515596.htm
https://www.chotot.com/mua-ban-cho-quan-8-tp-ho-chi-minh/116669977.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/117515356.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/117514979.htm
https://www.chotot.com/mua-ban-cho-huyen-cu-chi-tp-ho-chi-minh/117514867.htm
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/116569568.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/116886493.htm
https://www.chotot.com/mua-ban-cho-quan-binh-thanh-tp-ho-chi-minh/117513589.htm
https://www.chotot.com/mua-ban-cho-quan-phu-nhuan-tp-ho-chi-minh/117513415.htm
https://www.chotot.com/mua-ban-cho-huyen-nha-be-tp-ho-chi-minh/116966690.htm
https://www.chotot.com/mua-ban-cho-tp-ho-chi-minh?page=17
https://w

https://www.chotot.com/mua-ban-cho-huyen-hoc-mon-tp-ho-chi-minh/97116762.htm
https://www.chotot.com/mua-ban-cho-quan-3-tp-ho-chi-minh/117496948.htm
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/117496771.htm
https://www.chotot.com/mua-ban-cho-quan-1-tp-ho-chi-minh/117496582.htm
https://www.chotot.com/mua-ban-cho-quan-4-tp-ho-chi-minh/117496498.htm
https://www.chotot.com/mua-ban-cho-quan-8-tp-ho-chi-minh/117433487.htm
https://www.chotot.com/mua-ban-cho-quan-tan-phu-tp-ho-chi-minh/117496204.htm
https://www.chotot.com/mua-ban-cho-quan-8-tp-ho-chi-minh/117492876.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/117442116.htm
https://www.chotot.com/mua-ban-cho-tp-ho-chi-minh?page=22
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/117495460.htm
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/114844572.htm
https://www.chotot.com/mua-ban-cho-quan-8-tp-ho-chi-minh/117228793.htm
https://www.chotot.com/mua-ban-cho-quan-8-

...

https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/116400463.htm
https://www.chotot.com/mua-ban-cho-tp-ho-chi-minh?page=105
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/116399823.htm
https://www.chotot.com/mua-ban-cho-quan-phu-nhuan-tp-ho-chi-minh/116399494.htm
https://www.chotot.com/mua-ban-cho-quan-1-tp-ho-chi-minh/116398867.htm
https://www.chotot.com/mua-ban-cho-quan-8-tp-ho-chi-minh/116398720.htm
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/116397862.htm
https://www.chotot.com/mua-ban-cho-huyen-hoc-mon-tp-ho-chi-minh/116396974.htm
https://www.chotot.com/mua-ban-cho-quan-go-vap-tp-ho-chi-minh/116162688.htm
https://www.chotot.com/mua-ban-cho-quan-8-tp-ho-chi-minh/116394362.htm
https://www.chotot.com/mua-ban-cho-quan-8-tp-ho-chi-minh/116394121.htm
https://www.chotot.com/mua-ban-cho-quan-5-tp-ho-chi-minh/116393866.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/116391129.htm
https://www.chotot.com/mua-ban-cho-quan-tan-binh-

Problem reading geckodriver versions: error sending request for url (https://raw.githubusercontent.com/SeleniumHQ/selenium/trunk/common/geckodriver/geckodriver-support.json). Using latest geckodriver version
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


https://www.chotot.com/mua-ban-cho-huyen-nha-be-tp-ho-chi-minh/116364894.htm
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/116363650.htm
https://www.chotot.com/mua-ban-cho-huyen-binh-chanh-tp-ho-chi-minh/116363100.htm
https://www.chotot.com/mua-ban-cho-tp-ho-chi-minh?page=107
https://www.chotot.com/mua-ban-cho-quan-phu-nhuan-tp-ho-chi-minh/116362954.htm
https://www.chotot.com/mua-ban-cho-huyen-binh-chanh-tp-ho-chi-minh/116362424.htm
https://www.chotot.com/mua-ban-cho-huyen-hoc-mon-tp-ho-chi-minh/116361067.htm
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/116360611.htm
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/116355514.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/116354350.htm
https://www.chotot.com/mua-ban-cho-huyen-nha-be-tp-ho-chi-minh/116259649.htm
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/116351609.htm
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/116350322.htm
https://www.chotot.com/mua

https://www.chotot.com/mua-ban-cho-tp-ho-chi-minh?page=112
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/116245554.htm
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/116245965.htm
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/101745888.htm
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/109830785.htm
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/101677707.htm
https://www.chotot.com/mua-ban-cho-huyen-hoc-mon-tp-ho-chi-minh/116245431.htm
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/109489536.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/116242532.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/116241765.htm
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/116166189.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/116240929.htm
https://www.chotot.com/mua-ban-cho-huyen-binh-chanh-tp-ho-chi-minh/116240431.ht

https://www.chotot.com/mua-ban-cho-quan-7-tp-ho-chi-minh/116116532.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/116115794.htm
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/116113427.htm
https://www.chotot.com/mua-ban-cho-quan-4-tp-ho-chi-minh/116113153.htm
https://www.chotot.com/mua-ban-cho-huyen-cu-chi-tp-ho-chi-minh/116112613.htm
https://www.chotot.com/mua-ban-cho-thanh-pho-thu-duc-tp-ho-chi-minh/116111778.htm
https://www.chotot.com/mua-ban-cho-quan-3-tp-ho-chi-minh/116111291.htm
https://www.chotot.com/mua-ban-cho-quan-binh-thanh-tp-ho-chi-minh/116107873.htm
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/101139108.htm
https://www.chotot.com/mua-ban-cho-quan-binh-tan-tp-ho-chi-minh/100864381.htm
https://www.chotot.com/mua-ban-cho-huyen-binh-chanh-tp-ho-chi-minh/116105061.htm
https://www.chotot.com/mua-ban-cho-quan-phu-nhuan-tp-ho-chi-minh/116104289.htm
https://www.chotot.com/mua-ban-cho-quan-12-tp-ho-chi-minh/103786935.htm


In [5]:
df.to_csv('Dog.csv', index = False)
df.head(10)

,post_time,price,description,breed,age,size,address
0,Tin bán chuyên đăng 4 giờ trước,4.000.000 đ,Cần tìm nhà mới cho đàn Pomeranian nhà đẻ thuầ...,Chó Phốc Sóc,Chó con (dưới 3 tháng tuổi),Siêu nhỏ (tiny,Số 1 đường Bác Ái phường Bình Thọ quận Thủ Đức...
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Tin bán chuyên đăng 1 giờ trước,4.000.000 đ,Chào cả nhà hôm nay em lên cho cả nhà những bé...,Chó Phốc Sóc,Chó con (dưới 3 tháng tuổi),Siêu nhỏ (tiny,"Phường 1, Quận 8, Tp Hồ Chí Minh"
0,Tin bán chuyên đăng 1 giờ trước,2.000.000 đ,"Poodlle thuần chủng size tiny\nMặt ngắn, xinh ...",Chó Poodle,Chó con (dưới 3 tháng tuổi),Siêu nhỏ (tiny,"Phường Bình Hưng Hòa, Quận Bình Tân, Tp Hồ Chí..."
0,Tin bán chuyên đăng 2 giờ trước,2.000.000 đ,Chào cả nhà hôm nay em lên cho cả nhà những bé...,Chó Poodle,Chó con (dưới 3 tháng tuổi),Siêu nhỏ (tiny,"Phường 1, Quận 8, Tp Hồ Chí Minh"
0,Tin bán chuyên đăng 3 phút trước,3.000.000 đ,Chào cả nhà hôm nay em lên cho cả nhà những bé...,Chó Poodle,Chó con (dưới 3 tháng tuổi),Siêu nhỏ (tiny,"Phường An Lạc, Quận Bình Tân, Tp Hồ Chí Minh"
0,Tin bán chuyên đăng 2 phút trước,5.000.000 đ,Chào cả nhà hôm nay em lên cho cả nhà những bé...,Chó Phốc Sóc,Chó con (dưới 3 tháng tuổi),Siêu nhỏ (tiny,"Phường 1, Quận 8, Tp Hồ Chí Minh"
0,Tin cá nhân đăng 7 phút trước,12.000.000 đ,"Chó godenl thuần chủng , 12 tháng tuổi . Ngoa...",Chó Golden,Chó trưởng thành (hơn 1 tuổi),Trung bình,"Bàu Tre 1 Tân An Hội, Xã Tân An Hội, Huyện Củ ..."
0,Tin bán chuyên đăng 10 phút trước,1.200.000 đ,Tìm nhà mới cho mấy bé như hình \nĂn Uống khoẻ...,Khác,Chó con (dưới 3 tháng tuổi),Lớn,"Phường Thạnh Xuân, Quận 12, Tp Hồ Chí Minh"
0,Tin bán chuyên đăng 15 phút trước,4.000.000 đ,☘️Em lại có đàn Gấu xù Bắc Hà bụ bẫm gần 6kg.\...,Chó Gấu Bắc Hà,Chó con (dưới 3 tháng tuổi),Trung bình,"676 Lê Quang Định, Phường 1, Quận Gò Vấp, Tp H..."
